# Двоичные деревья поиска, АВЛ-деревья

In [1]:
import random
import sys

sys.setrecursionlimit(20000)

## 1. Двоичное дерево поиска

### Реализация

In [2]:
class NodeBinary:
    
    def __init__(self, value):
        self.value = value
        self.parent = None
        self.l_child = None
        self.r_child = None
        
    def is_l_child(self):
        return self.parent.l_child == self
    
    def is_r_child(self):
        return self.parent.r_child == self
    
    def is_leaf(self):
        return not self.has_l_child and not self.has_r_child()
    
    def has_l_child(self):
        return self.l_child is not None
    
    def has_r_child(self):
        return self.r_child is not None
        
    def insert_node(self, node):
        # если значение в текущем узле не больше чем значение в добавляемом узле
        if self.value < node.value:
            # если у текущего узла есть правый потомок
            if self.has_r_child():
                # добавляем узел в поддерево, где корень - правый потомок текущего узла
                self.r_child.insert_node(node)
            # если у текущего узла нет правого потомка
            else:
                # делаем добавляемый узел правым потомком текущего узла
                self.r_child = node
                # задаем у правого потомка ссылку на родителя, которым является текущий узел
                self.r_child.parent = self
        # если значение в текущем узле больше чем добавляемое значение
        if self.value >= node.value:
            # если у текущего узла есть левый потомок
            if self.has_l_child():
                # добавляем узел в поддерево, где корень - левый потомок текущего узла
                self.l_child.insert_node(node)
            # если у текущего узла нет левого потомка
            else:
                # делаем добавляемый узел левым потомком текущего узла
                self.l_child = node
                # задаем у левого потомка ссылку на родителя, которым является текущий узел
                self.l_child.parent = self
                
    def get_node(self, value):
        if self.value == value:
            return self
        elif self.value < value and self.has_r_child():
            return self.r_child.get_node(value)
        elif self.value > value and self.has_l_child():
            return self.l_child.get_node(value)
        
    def remove_node(self):
        # если удаляемый узел является левым потомком своего родителя
        if self.is_l_child():
            # обнуляем у родителя ссылку на левого потомка
            self.parent.l_child = None
        # если правым
        else:
            # обнуляем у родителя ссылку на правого потомка
            self.parent.r_child = None
        # если у удаляемого узла есть левый потомок
        if self.has_l_child():
            # добавляем его в дерево, где корнем является родитель удаляемого узла
            self.parent.insert_node(self.l_child)
        # если у удаляемого узла есть правый потомок
        if self.has_r_child():
            # добавляем его в дерево, где корнем является родитель удаляемого узла
            self.parent.insert_node(self.r_child)
            
    def get_new_root(self):
        if self.has_l_child() and not self.has_r_child():
            root = self.l_child
            self.l_child = None
        elif not self.has_l_child() and self.has_r_child():
            root = self.r_child
            self.r_child = None
        elif self.has_l_child() and self.has_r_child():
            root = self.l_child
            self.l_child = None
            root.insert_node(self.r_child)
            self.r_child = None
        else:
            return
        root.parent = None
        return root
        
    def __iter__(self):
        if self.has_l_child():
            for value in self.l_child:
                yield value
        yield self.value
        if self.has_r_child():
            for value in self.r_child:
                yield value

In [3]:
class TreeBinary:
    
    def __init__(self):
        self.root = None
        self.node_type = NodeBinary
        
    def insert(self, value):
        node = self.node_type(value)
        if self.root is None:
            self.root = node
        else:
            self.root.insert_node(node)
        return node
        
    def has_value(self, value):
        node = self.root.get_node(value)
        if node is not None:
            return True
        return False
    
    def remove(self, value):
        node = self.root.get_node(value)
        if node is None:
            print(f'Number {value} is not in the tree')
            return
        if node.parent is not None:
            node.remove_node()
            return node.parent
        else:
            self.root = self.root.get_new_root()
            
    def __iter__(self):
        for element in self.root:
            yield element

### Вспомогательные функции

In [4]:
def get_test_arrays(N, perc=0.1):
    array = [i for i in range(N)]
    random.shuffle(array)
    array_missing = [random.randint(N+1, N*2) for i in range(int(N*perc))]
    array_part = array[:int(N*perc)]
    random.shuffle(array_part)
    return array, array_missing, array_part

def print_tree_info(tree):
    numbers = list(tree)
    print('Количество узлов:', len(numbers))
    print('Обход дерева:', numbers[:15], '...')

# для сбора замеров времени в один массив
from IPython import get_ipython
from IPython.core.magics.execution import TimeitResult

RESULTS = []

def collect_results():
    for k, v in get_ipython().__dict__['user_ns']['Out'].items():
        if v.__class__ == TimeitResult:
            RESULTS.append(round(v.worst, 3))

### Тестирование времени работы двоичного дерева при добавлении отсортированных чисел

In [5]:
array_rand, array_miss, array_part = get_test_arrays(10000, perc=0.2)

In [6]:
tree_binary_sorted = TreeBinary()

#### Добавление отсортированных чисел

In [7]:
%%timeit -r 1 -n 1 -o -q

for i in sorted(array_rand):
    tree_binary_sorted.insert(i)

<TimeitResult : 23.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Обход дерева

In [8]:
%%timeit -r 1 -n 1 -o -q

print_tree_info(tree_binary_sorted)

Количество узлов: 10000
Обход дерева: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] ...


<TimeitResult : 3.18 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Поиск отсутствующих чисел

In [9]:
%%timeit -r 1 -n 1 -o -q

for i in array_miss:
    if tree_binary_sorted.has_value(i):
        print(f'{i} is in the tree')
        break

<TimeitResult : 8.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Поиск присутствующих чисел

In [10]:
%%timeit -r 1 -n 1 -o -q

for i in array_part:
    if not tree_binary_sorted.has_value(i):
        print(f'{i} is not in the tree')
        break

<TimeitResult : 3.68 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Удаление части имеющихся чисел

In [11]:
%%timeit -r 1 -n 1 -o -q

for i in array_part:
    tree_binary_sorted.remove(i)

<TimeitResult : 3.22 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Обход дерева

In [12]:
%%timeit -r 1 -n 1 -o -q

print_tree_info(tree_binary_sorted)

Количество узлов: 8000
Обход дерева: [0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17] ...


<TimeitResult : 1.52 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

### Тестирование времени работы двоичного дерева при добавлении чисел в случайном порядке

In [13]:
array_rand, array_miss, array_part = get_test_arrays(40000, perc=0.2)

In [14]:
tree_binary_random = TreeBinary()

#### Добавление чисел в случайном порядке

In [15]:
%%timeit -r 1 -n 1 -o -q

for i in array_rand:
    tree_binary_random.insert(i)

<TimeitResult : 293 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Обход дерева

In [16]:
%%timeit -r 1 -n 1 -o -q

print_tree_info(tree_binary_random)

Количество узлов: 40000
Обход дерева: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] ...


<TimeitResult : 57.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Поиск отсутствующих чисел

In [17]:
%%timeit -r 1 -n 1 -o -q

for i in array_miss:
    if tree_binary_random.has_value(i):
        print(f'{i} is in the tree')
        break

<TimeitResult : 25 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Поиск присутствующих чисел

In [18]:
%%timeit -r 1 -n 1 -o -q

for i in array_part:
    if not tree_binary_random.has_value(i):
        print(f'{i} is not in the tree')
        break

<TimeitResult : 43.6 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Удаление части имеющихся чисел

In [19]:
%%timeit -r 1 -n 1 -o -q

for i in array_part:
    tree_binary_random.remove(i)

<TimeitResult : 4.92 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Обход дерева

In [20]:
%%timeit -r 1 -n 1 -o -q

print_tree_info(tree_binary_random)

Количество узлов: 32000
Обход дерева: [1, 2, 3, 4, 5, 6, 7, 8, 10, 13, 14, 15, 16, 18, 19] ...


<TimeitResult : 19 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

## 2. АВЛ-дерево

### Реализация

In [21]:
class NodeAVL(NodeBinary):
    
    def __init__(self, value):
        super().__init__(value)
        self.height = 0

In [22]:
class TreeAVL(TreeBinary):
    
    def __init__(self):
        super().__init__()
        self.node_type = NodeAVL
    
    @staticmethod
    def get_height(node):
        if node is None:
            return 0
        return node.height
    
    def update_height(self, node):
        node.height = max(self.get_height(node.l_child), self.get_height(node.r_child)) + 1
    
    def insert(self, value):
        node = super().insert(value)
        self.balance(node)
        
    def remove(self, value):
        node = super().remove(value)
        self.balance(node)
    
    def rotate_right(self, node):
        #
        #      A           B
        #     / \         / \
        #    B  R  --->  L  A 
        #   / \            / \
        #  L  C           C  R 
        # 
        # A - node/old root, B - node.l_child/new root
        #
        B = node.l_child
        C = B.r_child
        node.l_child = C
        B.r_child = node
        B.parent = node.parent
        if node.parent is not None:
            if node.is_l_child():
                node.parent.l_child = B
            elif node.is_r_child():
                node.parent.r_child = B
        else:
            self.root = B
        node.parent = B
        if C is not None:
            C.parent = node
        self.update_height(node)
        self.update_height(B)
        
    def rotate_left(self, node):
        #
        #     A            B
        #    / \          / \
        #   L  B   --->  A  R 
        #     / \       / \
        #    C  R      L  C 
        # 
        # A - node/old root, B - node.r_child/new root
        #
        B = node.r_child
        C = B.l_child
        node.r_child = C
        B.l_child = node
        B.parent = node.parent
        if node.parent is not None:
            if node.is_l_child():
                node.parent.l_child = B
            elif node.is_r_child():
                node.parent.r_child = B
        else:
            self.root = B
        node.parent = B
        if C is not None:
            C.parent = node
        self.update_height(node)
        self.update_height(B)
        
    def balance(self, node):
        while node is not None:
            self.update_height(node)
            node_balance = self.get_height(node.l_child) - self.get_height(node.r_child)
            if node_balance > 1:
                if self.get_height(node.l_child.l_child) >= self.get_height(node.l_child.r_child):
                    self.rotate_right(node)
                else:
                    self.rotate_left(node.l_child)
                    self.rotate_right(node)
            elif node_balance < -1:
                if self.get_height(node.r_child.r_child) >= self.get_height(node.r_child.l_child):
                    self.rotate_left(node)
                else:
                    self.rotate_right(node.r_child)
                    self.rotate_left(node)
            node = node.parent   

### Тестирование времени работы АВЛ-дерева при добавлении отсортированных чисел

In [23]:
array_rand, array_miss, array_part = get_test_arrays(40000, perc=0.2)

In [24]:
tree_avl_sorted = TreeAVL()

#### Добавление отсортированных чисел

In [25]:
%%timeit -r 1 -n 1 -o -q

for i in sorted(array_rand):
    tree_avl_sorted.insert(i)

<TimeitResult : 849 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Обход дерева

In [26]:
%%timeit -r 1 -n 1 -o -q

print_tree_info(tree_avl_sorted)

Количество узлов: 40000
Обход дерева: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] ...


<TimeitResult : 37.9 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Поиск отсутствующих чисел

In [27]:
%%timeit -r 1 -n 1 -o -q

for i in array_miss:
    if tree_avl_sorted.has_value(i):
        print(f'{i} is in the tree')
        break

<TimeitResult : 43.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Поиск присутствующих чисел

In [28]:
%%timeit -r 1 -n 1 -o -q

for i in array_part:
    if not tree_avl_sorted.has_value(i):
        print(f'{i} is not in the tree')
        break

<TimeitResult : 47.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Удаление части имеющихся чисел

In [29]:
%%timeit -r 1 -n 1 -o -q

for i in array_part:
    tree_avl_sorted.remove(i)

<TimeitResult : 170 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Обход дерева

In [30]:
%%timeit -r 1 -n 1 -o -q

print_tree_info(tree_avl_sorted)

Количество узлов: 32000
Обход дерева: [0, 1, 2, 3, 4, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18] ...


<TimeitResult : 41.6 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

### Тестирование времени работы АВЛ-дерева при добавлении чисел в случайном порядке

In [31]:
array_rand, array_miss, array_part = get_test_arrays(40000, perc=0.2)

In [32]:
tree_avl_random = TreeAVL()

#### Добавление чисел

In [33]:
%%timeit -r 1 -n 1 -o -q

for i in array_rand:
    tree_avl_random.insert(i)

<TimeitResult : 855 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Обход дерева

In [34]:
%%timeit -r 1 -n 1 -o -q

print_tree_info(tree_avl_random)

Количество узлов: 40000
Обход дерева: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] ...


<TimeitResult : 48.9 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Поиск отсутствующих чисел

In [35]:
%%timeit -r 1 -n 1 -o -q

for i in array_miss:
    if tree_avl_random.has_value(i):
        print(f'{i} is in the tree')
        break

<TimeitResult : 30.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Поиск присутствующих чисел

In [36]:
%%timeit -r 1 -n 1 -o -q

for i in array_part:
    if not tree_avl_random.has_value(i):
        print(f'{i} is not in the tree')
        break

<TimeitResult : 46.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Удаление части имеющихся чисел

In [37]:
%%timeit -r 1 -n 1 -o -q

for i in array_part:
    tree_avl_random.remove(i)

<TimeitResult : 233 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

#### Обход дерева

In [38]:
%%timeit -r 1 -n 1 -o -q

print_tree_info(tree_avl_random)

Количество узлов: 32000
Обход дерева: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 17] ...


<TimeitResult : 39.6 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

## 3. Выводы

#### Таблица производительности при добавлении и поиске элементов

|Тип дерева|Тип массива|Количество элементов|Добавление|Поиск отсутствующих|Поиск присутствующих|
|:--------:|---------|--------------|--------------------|------------------|---|
|Бинарное|Отсортированный|10000|23.389|8.7|3.675|
|Бинарное|Случайный|40000|0.293|0.025|0.044|
|АВЛ|Отсортированный|40000|0.849|0.043|0.048|
|АВЛ|Случайный|40000|0.855|0.031|0.046|

#### Таблица производительности при удалении элементов и обходе дерева

|Тип дерева|Тип массива|Количество элементов|Удаление|Обход до удаления|Обход после удаления|
|:--------:|---------|--------------|--------------------|------------------|------|
|Бинарное|Отсортированный|10000|3.217,|3.18|1.523|
|Бинарное|Случайный|40000|4.92|0.058|19.044|
|АВЛ|Отсортированный|40000|0.17|0.038|0.042|
|АВЛ|Случайный|40000|0.233|0.049|0.04|

* Добавление элементов в дерево занимает наибольшее время в случае бинарного дерева и сортированного входного массива: дерево вырождается в связанный список с длиной равной длине входного массива. По этой же причине поиск и удаление элементов максимально по сравнению с другими случаями.
* Случайно перемешанный входной массив позволяет быстро добавлять и искать элементы в обычном бинарное дерево. Удаление элементов и операции, производимые после этого, уже занимают больше времени по сравнению с АВЛ-деревом. Из-за того, что при удалении не происходит балансировки, структура дерева препятствует эффективному выполнению операций.
* В случае АВЛ-дерева наличие или отсутствие сортировки входного массива не влияет на совершаемые операции - балансировка позволяет выстроить эффективную структуру дерева.

In [ ]:
collect_results()

In [43]:
for i in range(4):
    print(RESULTS[i*6:6*(i+1)])

[23.389, 3.18, 8.7, 3.675, 3.217, 1.523]
[0.293, 0.058, 0.025, 0.044, 4.92, 19.044]
[0.849, 0.038, 0.043, 0.048, 0.17, 0.042]
[0.855, 0.049, 0.031, 0.046, 0.233, 0.04]
